<a href="https://colab.research.google.com/github/KimJisanER/code_jjambbong/blob/main/0815_metabolism_dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [3]:
# seed
seed = 42

# define rmse
rmse = lambda x, y: np.mean((x - y) ** 2) ** 0.5

# features and targets
features = ["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]
mlm_target = "MLM"
hlm_target = "HLM"

# load data
df = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
df["AlogP"] = np.where(pd.isna(df["AlogP"]), df["LogD"], df["AlogP"])

# train
scores = []
reg_mlms = []
reg_hlms = []
kf = KFold(n_splits = 10, random_state = seed, shuffle = True)
for i, (train_index, valid_index) in enumerate(kf.split(df)):
    df_train = df.iloc[train_index]
    df_valid = df.iloc[valid_index]

    x_train = df_train[features].values
    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid = df_valid[features].values
    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    reg_mlm = RandomForestRegressor(random_state = seed)
    reg_mlm.fit(x_train, y_mlm_train)
    p_mlm = reg_mlm.predict(x_valid)

    reg_hlm = RandomForestRegressor(random_state = seed)
    reg_hlm.fit(x_train, y_hlm_train)
    p_hlm = reg_hlm.predict(x_valid)

    score = 0.5 * rmse(y_mlm_valid, p_mlm) + 0.5 * rmse(y_hlm_valid, p_hlm)

    reg_mlms.append(reg_mlm)
    reg_hlms.append(reg_hlm)
    scores.append(score)
    print(f"Fold {i+1:2d}: {score:.5f}")

print(f"CV score: {np.mean(scores):.5f}")

Fold  1: 34.16658
Fold  2: 34.03305
Fold  3: 33.21484
Fold  4: 34.63931
Fold  5: 32.44617
Fold  6: 33.18904
Fold  7: 31.83381
Fold  8: 31.91317
Fold  9: 31.68066
Fold 10: 33.10630
CV score: 33.02229


In [4]:
# load data
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

train_step2 = train[['id', 'SMILES', 'MLM', 'HLM']]

# predict
train_step2["pre_MLM"] = np.mean([reg_mlm.predict(train[features].values) for reg_mlm in reg_mlms], axis = 0)
train_step2["pre_HLM"] = np.mean([reg_hlm.predict(train[features].values) for reg_hlm in reg_hlms], axis = 0)

<ipython-input-4-51f764981192>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_step2["pre_MLM"] = np.mean([reg_mlm.predict(train[features].values) for reg_mlm in reg_mlms], axis = 0)
<ipython-input-4-51f764981192>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_step2["pre_HLM"] = np.mean([reg_hlm.predict(train[features].values) for reg_hlm in reg_hlms], axis = 0)


In [5]:
# load data
test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

test_step2 = test[['id', 'SMILES']]

# predict
test_step2["pre_MLM"] = np.mean([reg_mlm.predict(test[features].values) for reg_mlm in reg_mlms], axis = 0)
test_step2["pre_HLM"] = np.mean([reg_hlm.predict(test[features].values) for reg_hlm in reg_hlms], axis = 0)

<ipython-input-5-f1c274ce2d1c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_step2["pre_MLM"] = np.mean([reg_mlm.predict(test[features].values) for reg_mlm in reg_mlms], axis = 0)
<ipython-input-5-f1c274ce2d1c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_step2["pre_HLM"] = np.mean([reg_hlm.predict(test[features].values) for reg_hlm in reg_hlms], axis = 0)


In [6]:
test_step2

,id,SMILES,pre_MLM,pre_HLM
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,30.348972,46.114039
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,67.695643,79.545362
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,28.258084,41.327114
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,44.590508,75.296760
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,55.058119,77.867187
...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,10.594797,26.257306
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,72.272508,82.527776
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,38.044879,76.583595
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,61.165179,78.091850


In [7]:
train_step2['diff_MLM'] = train_step2['MLM'] - train_step2['pre_MLM']
train_step2['diff_HLM'] = train_step2['HLM'] - train_step2['pre_HLM']

<ipython-input-7-f77147487caa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_step2['diff_MLM'] = train_step2['MLM'] - train_step2['pre_MLM']
<ipython-input-7-f77147487caa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_step2['diff_HLM'] = train_step2['HLM'] - train_step2['pre_HLM']


In [8]:
train_step2

,id,SMILES,MLM,HLM,pre_MLM,pre_HLM,diff_MLM,diff_HLM
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,26.087607,48.229084,-0.077607,2.450916
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,36.723498,63.144570,-7.453498,-12.554570
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,24.392497,73.165869,-18.806497,7.726131
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,15.301428,14.598503,-9.591428,-12.598503
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,70.904511,82.824144,22.365489,17.165856
...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,6.790676,12.693100,-5.234676,-9.614100
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,37.278186,59.208126,-1.718186,-11.578126
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,45.451169,22.864715,10.698831,-21.074715
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,19.500675,31.541213,-19.470675,-28.771213


In [9]:
data_merge = pd.concat([train_step2, test_step2], axis=0).reset_index(drop=True)

In [10]:
data_merge

,id,SMILES,MLM,HLM,pre_MLM,pre_HLM,diff_MLM,diff_HLM
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,26.087607,48.229084,-0.077607,2.450916
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,36.723498,63.144570,-7.453498,-12.554570
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,24.392497,73.165869,-18.806497,7.726131
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,15.301428,14.598503,-9.591428,-12.598503
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,70.904511,82.824144,22.365489,17.165856
...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,10.594797,26.257306,NaN,NaN
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,72.272508,82.527776,NaN,NaN
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,38.044879,76.583595,NaN,NaN
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,61.165179,78.091850,NaN,NaN


In [11]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem.Draw import IPythonConsole

In [12]:
fp_list = []
for idx, smiles in enumerate(data_merge["SMILES"]): # Smiles 열에서 반복.
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits = 1024)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp.ToBitString()])

dic = {}
for idx in range(1024):
  key = "F" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(1024):
    key = "F" + str(idx)
    dic[key].append(fp[idx])

fp_data = pd.DataFrame(dic)

In [13]:
fp_data

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F1014,F1015,F1016,F1017,F1018,F1019,F1020,F1021,F1022,F1023
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3976,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3978,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3979,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data_fp = pd.concat([data_merge, fp_data], axis= 1)

In [15]:
data_fp

,id,SMILES,MLM,HLM,pre_MLM,pre_HLM,diff_MLM,diff_HLM,F0,F1,...,F1014,F1015,F1016,F1017,F1018,F1019,F1020,F1021,F1022,F1023
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,26.087607,48.229084,-0.077607,2.450916,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,36.723498,63.144570,-7.453498,-12.554570,0,0,...,0,0,0,0,0,1,0,0,0,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,24.392497,73.165869,-18.806497,7.726131,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,15.301428,14.598503,-9.591428,-12.598503,0,1,...,0,0,0,0,0,1,0,1,0,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,70.904511,82.824144,22.365489,17.165856,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,10.594797,26.257306,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,72.272508,82.527776,NaN,NaN,0,0,...,0,0,0,1,0,0,0,0,0,0
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,38.044879,76.583595,NaN,NaN,0,1,...,0,0,0,0,0,1,0,0,0,0
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,61.165179,78.091850,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
train_df = data_fp.iloc[:3498]
test_df = data_fp.iloc[3498:]

In [17]:
train_df

,id,SMILES,MLM,HLM,pre_MLM,pre_HLM,diff_MLM,diff_HLM,F0,F1,...,F1014,F1015,F1016,F1017,F1018,F1019,F1020,F1021,F1022,F1023
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,26.087607,48.229084,-0.077607,2.450916,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,36.723498,63.144570,-7.453498,-12.554570,0,0,...,0,0,0,0,0,1,0,0,0,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,24.392497,73.165869,-18.806497,7.726131,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,15.301428,14.598503,-9.591428,-12.598503,0,1,...,0,0,0,0,0,1,0,1,0,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,70.904511,82.824144,22.365489,17.165856,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,6.790676,12.693100,-5.234676,-9.614100,0,0,...,0,0,0,0,0,0,0,0,0,0
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,37.278186,59.208126,-1.718186,-11.578126,0,0,...,0,0,0,0,0,0,0,0,0,0
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,45.451169,22.864715,10.698831,-21.074715,1,0,...,0,0,0,0,0,0,0,0,0,0
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,19.500675,31.541213,-19.470675,-28.771213,0,0,...,0,0,0,0,0,1,0,0,0,0


In [18]:
test_df

,id,SMILES,MLM,HLM,pre_MLM,pre_HLM,diff_MLM,diff_HLM,F0,F1,...,F1014,F1015,F1016,F1017,F1018,F1019,F1020,F1021,F1022,F1023
3498,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,NaN,NaN,30.348972,46.114039,NaN,NaN,0,1,...,0,0,0,0,0,1,0,0,0,0
3499,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,NaN,NaN,67.695643,79.545362,NaN,NaN,0,0,...,0,0,0,1,0,0,0,0,0,0
3500,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,NaN,NaN,28.258084,41.327114,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3501,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,NaN,NaN,44.590508,75.296760,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3502,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,NaN,NaN,55.058119,77.867187,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,10.594797,26.257306,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,72.272508,82.527776,NaN,NaN,0,0,...,0,0,0,1,0,0,0,0,0,0
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,38.044879,76.583595,NaN,NaN,0,1,...,0,0,0,0,0,1,0,0,0,0
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,61.165179,78.091850,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# seed
seed = 42

# define rmse
rmse = lambda x, y: np.mean((x - y) ** 2) ** 0.5

# features and targets
non_features = ["id", "SMILES", "MLM", "HLM", "pre_MLM", "pre_HLM", "diff_MLM", "diff_HLM"]
features = [ column for column in train_df.columns if column not in non_features]
mlm_target = "diff_MLM"
hlm_target = "diff_HLM"

# train
scores = []
reg_mlms = []
reg_hlms = []
kf = KFold(n_splits = 10, random_state = seed, shuffle = True)
for i, (train_index, valid_index) in enumerate(kf.split(train_df)):
    df_train = train_df.iloc[train_index]
    df_valid = train_df.iloc[valid_index]

    x_train = df_train[features].values
    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid = df_valid[features].values
    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    reg_mlm = RandomForestRegressor(random_state = seed)
    reg_mlm.fit(x_train, y_mlm_train)
    p_mlm = reg_mlm.predict(x_valid)

    reg_hlm = RandomForestRegressor(random_state = seed)
    reg_hlm.fit(x_train, y_hlm_train)
    p_hlm = reg_hlm.predict(x_valid)

    score = 0.5 * rmse(y_mlm_valid, p_mlm) + 0.5 * rmse(y_hlm_valid, p_hlm)

    reg_mlms.append(reg_mlm)
    reg_hlms.append(reg_hlm)
    scores.append(score)
    print(f"Fold {i+1:2d}: {score:.5f}")

print(f"CV score: {np.mean(scores):.5f}")

Fold  1: 14.46885


KeyboardInterrupt: ignored

In [37]:
test_df["diff_MLM"] = np.mean([reg_mlm.predict(test_df[features].values) for reg_mlm in reg_mlms], axis = 0)
test_df["diff_HLM"] = np.mean([reg_hlm.predict(test_df[features].values) for reg_hlm in reg_hlms], axis = 0)

<ipython-input-37-01e4ffc47077>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["diff_MLM"] = np.mean([reg_mlm.predict(test_df[features].values) for reg_mlm in reg_mlms], axis = 0)
<ipython-input-37-01e4ffc47077>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["diff_HLM"] = np.mean([reg_hlm.predict(test_df[features].values) for reg_hlm in reg_hlms], axis = 0)


In [38]:
test_df

,id,SMILES,MLM,HLM,pre_MLM,pre_HLM,diff_MLM,diff_HLM,F0,F1,...,F1014,F1015,F1016,F1017,F1018,F1019,F1020,F1021,F1022,F1023
3498,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,NaN,NaN,30.348972,46.114039,-3.867940,-4.859926,0,1,...,0,0,0,0,0,1,0,0,0,0
3499,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,NaN,NaN,67.695643,79.545362,3.649865,2.184558,0,0,...,0,0,0,1,0,0,0,0,0,0
3500,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,NaN,NaN,28.258084,41.327114,2.494462,-0.997784,0,0,...,0,0,0,0,0,0,0,0,0,0
3501,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,NaN,NaN,44.590508,75.296760,-0.977068,0.185481,0,0,...,0,0,0,0,0,0,0,0,0,0
3502,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,NaN,NaN,55.058119,77.867187,-4.509563,2.796477,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,10.594797,26.257306,-0.870405,-2.482153,0,0,...,0,0,0,0,0,0,0,0,0,0
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,72.272508,82.527776,3.998502,2.200545,0,0,...,0,0,0,1,0,0,0,0,0,0
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,38.044879,76.583595,-2.776965,-2.533356,0,1,...,0,0,0,0,0,1,0,0,0,0
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,61.165179,78.091850,-2.726437,-0.992974,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
test_df['MLM'] = test_df['pre_MLM'] + test_df['diff_MLM']
test_df['HLM'] = test_df['pre_HLM'] + test_df['diff_HLM']

<ipython-input-39-03568c81f96c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['MLM'] = test_df['pre_MLM'] + test_df['diff_MLM']
<ipython-input-39-03568c81f96c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['HLM'] = test_df['pre_HLM'] + test_df['diff_HLM']


In [40]:
test_df

,id,SMILES,MLM,HLM,pre_MLM,pre_HLM,diff_MLM,diff_HLM,F0,F1,...,F1014,F1015,F1016,F1017,F1018,F1019,F1020,F1021,F1022,F1023
3498,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,26.481032,41.254113,30.348972,46.114039,-3.867940,-4.859926,0,1,...,0,0,0,0,0,1,0,0,0,0
3499,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,71.345508,81.729920,67.695643,79.545362,3.649865,2.184558,0,0,...,0,0,0,1,0,0,0,0,0,0
3500,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,30.752546,40.329330,28.258084,41.327114,2.494462,-0.997784,0,0,...,0,0,0,0,0,0,0,0,0,0
3501,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,43.613440,75.482241,44.590508,75.296760,-0.977068,0.185481,0,0,...,0,0,0,0,0,0,0,0,0,0
3502,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,50.548556,80.663664,55.058119,77.867187,-4.509563,2.796477,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,9.724392,23.775153,10.594797,26.257306,-0.870405,-2.482153,0,0,...,0,0,0,0,0,0,0,0,0,0
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,76.271010,84.728321,72.272508,82.527776,3.998502,2.200545,0,0,...,0,0,0,1,0,0,0,0,0,0
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,35.267914,74.050239,38.044879,76.583595,-2.776965,-2.533356,0,1,...,0,0,0,0,0,1,0,0,0,0
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,58.438742,77.098876,61.165179,78.091850,-2.726437,-0.992974,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
sub = test_df[['id', 'MLM', 'HLM']].reset_index(drop=True)

In [50]:
sub

,id,MLM,HLM
0,TEST_000,26.481032,41.254113
1,TEST_001,71.345508,81.729920
2,TEST_002,30.752546,40.329330
3,TEST_003,43.613440,75.482241
4,TEST_004,50.548556,80.663664
...,...,...,...
478,TEST_478,9.724392,23.775153
479,TEST_479,76.271010,84.728321
480,TEST_480,35.267914,74.050239
481,TEST_481,58.438742,77.098876


In [49]:
sub.to_csv('/content/drive/MyDrive/metabolism_dacon/0815_submission.csv', index=False)